In [ ]:
import os
import random
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset,SubsetRandomSampler
from sklearn.metrics import classification_report
from torchvision import transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns
from sklearn.metrics import precision_recall_curve,f1_score,roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_curve,f1_score,roc_curve,roc_auc_score,auc,accuracy_score,average_precision_score,precision_score,recall_score
from sklearn.preprocessing import label_binarize
import warnings
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings("ignore")
import torchvision
from torchvision import models
import torch.nn.functional as F
import torch.nn.init as init

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
def make_img(t_img):
    img = pd.read_pickle(t_img)
    img_l = []
    for i in range(len(img)):
        img_l.append(img.values[i][0])
    
    return np.array(img_l)

In [ ]:
def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight, 1)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LayerNorm):
        nn.init.constant_(m.weight, 1)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

In [ ]:
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
def plot_classification_report(y_tru, y_prd, mode, learning_rate, batch_size,epochs, figsize=(7, 7), ax=None):

    plt.figure(figsize=figsize)

    xticks = ['precision', 'recall', 'f1-score', 'support']
    yticks = ["Control", "Moderate", "Alzheimer's" ] 
    yticks += ['avg']

    rep = np.array(precision_recall_fscore_support(y_tru, y_prd)).T
    avg = np.mean(rep, axis=0)
    avg[-1] = np.sum(rep[:, -1])
    rep = np.insert(rep, rep.shape[0], avg, axis=0)

    sns.heatmap(rep,
                annot=True, 
                cbar=False, 
                xticklabels=xticks, 
                yticklabels=yticks,
                ax=ax, cmap = "Blues")
    
    plt.savefig('report_' + str(mode) + '_' + str(learning_rate) +'_' + str(batch_size)+'_' + str(epochs)+'.png')

In [ ]:
def calc_confusion_matrix(result, test_label,mode, learning_rate, batch_size, epochs):
    result = F.one_hot(result,num_classes=4)


    test_label = F.one_hot(test_label,num_classes=4)


    true_label= np.argmax(test_label, axis =1)

    predicted_label= np.argmax(result, axis =1)
    
    n_classes = 4
    precision = dict()
    recall = dict()
    thres = dict()
    for i in range(n_classes):
        precision[i], recall[i], thres[i] = precision_recall_curve(test_label[:, i],
                                                            result[:, i])


    print ("Classification Report :") 
    print (classification_report(true_label, predicted_label))
    cr = classification_report(true_label, predicted_label, output_dict=True)
    return cr, precision, recall, thres

In [ ]:
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


# 多尺度卷积块
class MultiScaleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(MultiScaleConv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2)
        self.conv5 = nn.Conv2d(in_channels, out_channels, kernel_size=7, padding=3)
        self.bn = nn.BatchNorm2d(out_channels * 3)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv3(x)
        x3 = self.conv5(x)
        x = torch.cat((x1, x2, x3), dim=1)
        x = self.bn(x)
        return x
    
class new_AttentionMultiScaleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(new_AttentionMultiScaleCNN, self).__init__()
        self.multi_scale_conv1 = MultiScaleConv(3, 64)
        self.se1 = SELayer(64 * 3)

        self.multi_scale_conv2 = MultiScaleConv(64 * 3, 100)
        self.se2 = SELayer(100 * 3)

        self.multi_scale_conv3 = MultiScaleConv(100 * 3, 50)
        self.se3 = SELayer(50 * 3)

        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(50 * 3 * 9 * 9, num_classes)

    def forward(self, x):
        x = self.relu(self.multi_scale_conv1(x))
        x = self.se1(x)
        x = self.maxpool(x)

        x = self.relu(self.multi_scale_conv2(x))
        x = self.se2(x)
        x = self.maxpool(x)

        x = self.relu(self.multi_scale_conv3(x))
        x = self.se3(x)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self,input,num_classes=4):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, num_classes)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.2)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.batch_norm3 = nn.BatchNorm1d(64)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

In [ ]:
class FC(nn.Module):
    def __init__(self, in_size, out_size, dropout_r=0., use_relu=True):
        super(FC, self).__init__()
        self.dropout_r = dropout_r
        self.use_relu = use_relu

        self.linear = nn.Linear(in_size, out_size)

        if use_relu:
            self.relu = nn.ReLU(inplace=True)

        if dropout_r > 0:
            self.dropout = nn.Dropout(dropout_r)

    def forward(self, x):
        x = self.linear(x)

        if self.use_relu:
            x = self.relu(x)

        if self.dropout_r > 0:
            x = self.dropout(x)

        return x

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim,output_dim, hidden_dims=[100,200,100]):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(nn.ReLU())

        for i in range(1, len(hidden_dims)):
            layers.append(nn.Linear(hidden_dims[i-1], hidden_dims[i]))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_dims[-1], output_dim))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
class FFN(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(FFN, self).__init__()

        self.mlp = MLP(
            input_dim=input_dim,
            output_dim=output_dim,
        )

    def forward(self, x):
        return self.mlp(x)

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self,in_dim):
        super(SelfAttention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=in_dim, num_heads=2)
        self.dropout1 = nn.Dropout(0.3)
        self.layer_norm1 = nn.LayerNorm(in_dim)
        self.dropout2 = nn.Dropout(0.2)
        self.layer_norm2 = nn.LayerNorm(in_dim)
        self.ffn=FFN(in_dim,in_dim)

    def forward(self, inputs):
        inputs=torch.unsqueeze(inputs, dim=1)
        outputs, _ = self.multihead_attn(inputs, inputs, inputs, attn_mask=None)
        outputs = self.dropout1(outputs)
        outputs = self.layer_norm1(inputs + outputs)
        outputs_2=self.ffn(outputs)
        outputs_2 = self.dropout2(outputs_2)
        all_outputs=self.layer_norm2(outputs + outputs_2)
        return all_outputs[:,0,:]

In [ ]:

class Cross_modal_transformer(nn.Module):
    def __init__(self):
        super(Cross_modal_transformer, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=50, num_heads=2)

        self.dropout1 = nn.Dropout(0.5)
        self.layer_norm = nn.LayerNorm(50)
        self.dropout2 = nn.Dropout(0.3)
        self.ffn=MLP(50,50,[50,100,50])

    def forward(self, x, y,layer_num):
        x=torch.unsqueeze(x, dim=1)
        y=torch.unsqueeze(y, dim=1)
        for i in range(layer_num):
            x = self.layer_norm(x)
            y = self.layer_norm(y)
            att_x = x + self.dropout1(self.multihead_attn(x, y, y, attn_mask=None)[0])
            att_y = y + self.dropout1(self.multihead_attn(y, x, x, attn_mask=None)[0])

            x = att_x + self.dropout2(self.ffn(self.layer_norm(att_x)))
            y = att_y + self.dropout2(self.ffn(self.layer_norm(att_x)))

        return x.squeeze(1),y.squeeze(1)

In [ ]:
class multi_modal_model(nn.Module):
    def __init__(self,mode,transcriptomicsinput):
        super(multi_modal_model, self).__init__()
        self.plasma_modal=NeuralNetwork(transcriptomicsinput,num_classes=50)

        self.img_modal=new_AttentionMultiScaleCNN(num_classes=50)

        self.NeuralNetwork=NeuralNetwork(transcriptomicsinput,num_classes=4)
        self.img=new_AttentionMultiScaleCNN(num_classes=4)
    
        self.mode=mode

        self.fc0 = nn.Linear(100, 4)
        self.fc2 = nn.Linear(200,4)
        self.fc0.apply(weight_init)
        self.fc2.apply(weight_init)

        self.transformer=Cross_modal_transformer()
        self.transformer.apply(weight_init)

        self.mlp1=MLP(100,4,[100,100,50])
        self.mlp1.apply(weight_init)

        self.mlp2=MLP(200,4,[200,100,50])
        self.mlp2.apply(weight_init)

    def forward(self, train_plasma, train_img):

        dense_plasma=self.plasma_modal(train_plasma)
        dense_img=self.img_modal(train_img)
        if self.mode == 'None':
            merged = torch.cat((dense_plasma,dense_img),axis=1)

            output = self.mlp1(merged)
        elif self.mode == 'MM_Tansformer':
            pp_att,ii_att = self.transformer(dense_plasma,dense_img,layer_num=1)
            
            merged = torch.cat((pp_att,ii_att),axis=1)
            output=self.mlp2(torch.cat((dense_plasma,dense_img,merged),axis=1))
        return dense_plasma,dense_img,torch.softmax(output,dim=1),merged

In [ ]:
class multi_modal_model_MDB(nn.Module):
    def __init__(self,mode,transcriptomicsinput):
        super(multi_modal_model_MDB, self).__init__()
        self.plasma_modal=NeuralNetwork(transcriptomicsinput,num_classes=4)
        self.plasma_modal.load_state_dict(torch.load('../../models/PPMI/Transcriptomics/best_146.pth'))
        self.plasma_modal.fc4 = nn.Linear(64, 50)

        self.img_modal=new_AttentionMultiScaleCNN(num_classes=4)
        self.img_modal.load_state_dict(torch.load('../../models/PPMI/MRI/best_22.pth'))
        self.img_modal.fc = nn.Linear(50 * 3 * 9 * 9, 50)

        self.NeuralNetwork=NeuralNetwork(transcriptomicsinput,num_classes=4)
        self.img=new_AttentionMultiScaleCNN(num_classes=4)
    
        self.mode=mode


        self.fc0 = nn.Linear(100, 4)
        self.fc2 = nn.Linear(200,4)
        self.fc0.apply(weight_init)
        self.fc2.apply(weight_init)

        self.transformer=Cross_modal_transformer()
        self.transformer.apply(weight_init)

        self.mlp1=MLP(100,4,[100,100,50])
        self.mlp1.apply(weight_init)

        self.mlp2=MLP(200,4,[200,100,50])
        self.mlp2.apply(weight_init)

    def forward(self, train_plasma, train_img):

        dense_plasma=self.plasma_modal(train_plasma)
        dense_img=self.img_modal(train_img)
        if self.mode == 'None':
            merged = torch.cat((dense_plasma,dense_img),axis=1)
            output = self.mlp1(merged)
            
        elif self.mode == 'MM_Tansformer':
            pp_att,ii_att = self.transformer(dense_plasma,dense_img,layer_num=5)
            
            merged = torch.cat((pp_att,ii_att),axis=1)
            output=self.mlp2(torch.cat((dense_plasma,dense_img,merged),axis=1))
        return dense_plasma,dense_img,torch.softmax(output,dim=1),merged

In [ ]:
def train(mode,MDB, batch_size, epochs, learning_rate, seed,a_alpha):
    alpha=a_alpha

    train_transcriptomics = pd.read_csv("../../processed_data/PPMI/overlap/X_train_transpot.csv").drop("PATNO Visit", axis=1).values
    val_transcriptomics = pd.read_csv("../../processed_data/PPMI/overlap/X_val_transpot.csv").drop("PATNO Visit", axis=1).values
    test_transcriptomics = pd.read_csv("../../processed_data/PPMI/overlap/X_test_transpot.csv").drop("PATNO Visit", axis=1).values
    
    train_img = make_img("../../processed_data/PPMI/overlap/X_train_img.pkl")
    val_img = make_img("../../processed_data/PPMI/overlap/X_val_img.pkl")
    test_img = make_img("../../processed_data/PPMI/overlap/X_test_img.pkl")
    train_label = pd.read_csv("../../processed_data/PPMI/overlap/y_train.csv").drop("PATNO Visit", axis=1).values.astype("int").flatten()
    val_label = pd.read_csv("../../processed_data/PPMI/overlap/y_val.csv").drop("PATNO Visit", axis=1).values.astype("int").flatten()
    test_label = pd.read_csv("../../processed_data/PPMI/overlap/y_test.csv").drop("PATNO Visit", axis=1).values.astype("int").flatten()

    train_transcriptomics_tensor = torch.tensor(train_transcriptomics, dtype=torch.float).to(device)
    val_transcriptomics_tensor = torch.tensor(val_transcriptomics, dtype=torch.float).to(device)
    test_transcriptomics_tensor = torch.tensor(test_transcriptomics, dtype=torch.float).to(device)

    train_img_tensor = torch.tensor(train_img, dtype=torch.float).to(device)
    val_img_tensor = torch.tensor(val_img, dtype=torch.float).to(device)
    test_img_tensor = torch.tensor(test_img, dtype=torch.float).to(device)

    train_label_tensor = torch.tensor(train_label, dtype=torch.long).to(device)
    val_label_tensor = torch.tensor(val_label, dtype=torch.long).to(device)
    test_label_tensor = torch.tensor(test_label, dtype=torch.long).to(device)

    train_dataset = TensorDataset(train_transcriptomics_tensor,train_img_tensor, train_label_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataset = TensorDataset(val_transcriptomics_tensor,val_img_tensor, val_label_tensor)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_dataset = TensorDataset(test_transcriptomics_tensor,test_img_tensor, test_label_tensor)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


    class_weights = compute_class_weight('balanced', classes=torch.unique(torch.tensor(train_label)).numpy(), y=train_label)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    # print(class_weights)

    # Initialize model
    if MDB:
        print('MDB')
        model = multi_modal_model_MDB(mode,train_transcriptomics.shape[1]).to(device)
    else:
        model = multi_modal_model(mode,train_transcriptomics.shape[1]).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    best_val_acc=0
    ratio_list=[]
    for epoch in range(epochs):
        ratio_epoch=[]
        # set_random_seed(epoch)
        softmax = nn.Softmax(dim=1)
        relu = nn.ReLU(inplace=True)
        tanh = nn.Tanh()
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        predicted_label=[]
        true_label=[]
        for transpot,img, labels in train_loader:
            optimizer.zero_grad()
            dense_transpot,dense_img,outputs,_ = model(transpot,img)

            if mode=='None':
                weight_size = model.fc0.weight.size(1)
                out_transpot = (torch.mm(dense_transpot, torch.transpose(model.fc0.weight[:, :weight_size // 2], 0, 1))
                        + model.fc0.bias / 2)
                out_img = (torch.mm(dense_img, torch.transpose(model.fc0.weight[:, weight_size // 2:], 0, 1))
                        + model.fc0.bias / 2)
            else:
                weight_size = model.fc2.weight.size(1)
                out_transpot = (torch.mm(dense_transpot, torch.transpose(model.fc2.weight[:, :weight_size // 4], 0, 1))
                        + model.fc2.bias / 2)
                out_img = (torch.mm(dense_img, torch.transpose(model.fc2.weight[:, weight_size // 4:weight_size // 2], 0, 1))
                        + model.fc2.bias / 2)
            _, predicted_T = out_transpot.max(1)
            _, predicted_M = out_img.max(1)
            score_transcriptomics = torch.tensor(f1_score(labels.tolist(),predicted_T.tolist(),average='macro'))
            score_img = torch.tensor(f1_score(labels.tolist(),predicted_M.tolist(),average='macro'))

            
            
            ratio_transpot = score_transcriptomics / score_img
            ratio_img = 1/ratio_transpot
            loss_transcriptomics = criterion(out_transpot, labels)
            loss_img = criterion(out_img, labels)

            ratio_epoch.append(ratio_img.cpu().detach().numpy())

            if MDB:
                if ratio_transpot > 1:
                    beta = 0  # audio coef
                    lam = tanh(alpha *relu(ratio_transpot))  # visual coef
                elif ratio_transpot < 1:
                    beta = tanh(alpha *relu(ratio_img))
                    lam = 0
                else:
                    beta = 0
                    lam = 0
                loss = criterion(outputs, labels) + beta * loss_transcriptomics + lam * loss_img
                loss.backward()
            else:
                loss = criterion(outputs, labels)
                loss.backward()
        

            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)

            total += labels.size(0)
            # correct += predicted.eq(labels).sum().item()
            predicted_label.extend(predicted.tolist())

            true_label.extend(labels.tolist())
        # print(sum(ratio_epoch)/len(ratio_epoch))
        ratio_list.append(sum(ratio_epoch)/len(ratio_epoch))

        train_loss = running_loss / len(train_loader)
        # train_acc = 100.0 * correct / total
        train_acc = f1_score(true_label,predicted_label,average='macro')
        
        # 在验证集上评估模型
        model.eval()
        predicted_label=[]
        # true_label=[]
        with torch.no_grad():
            _,_,outputs,_ = model(val_transcriptomics_tensor,val_img_tensor)
            # probs = nn.functional.softmax(outputs, dim=1)
            # all_probs.extend(probs.cpu().detach().numpy())
            _, predicted = outputs.max(1)
            predicted_label.extend(predicted.tolist())
        # val_acc = 100.0 * correct / total
        val_acc = f1_score(val_label,predicted_label,average='macro')
        predicted_label=[]
        with torch.no_grad():
            _,_,outputs,_ = model(test_transcriptomics_tensor,test_img_tensor)
            # probs = nn.functional.softmax(outputs, dim=1)
            # all_probs.extend(probs.cpu().detach().numpy())
            _, predicted = outputs.max(1)
            predicted_label.extend(predicted.tolist())
        # test_acc = 100.0 * correct / total
        test_acc = f1_score(test_label,predicted_label,average='macro')
        print(f"Epoch {epoch+1}, train Loss: {train_loss},train acc:{train_acc},val acc:{val_acc},best acc:{best_val_acc},test acc:{test_acc},ρ:{sum(ratio_epoch)/len(ratio_epoch)}")
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = model
            if MDB:
                torch.save(best_model.state_dict(), f'../../models/PPMI/Multi-modal/{mode}_MDB_{alpha}_best_{seed}.pth')
            else:
                torch.save(best_model.state_dict(), f'../../models/PPMI/Multi-modal/{mode}_best_{seed}.pth')
            print(f'Epoch {epoch+1} get best modal')
    
        
    if MDB:
        test_model=multi_modal_model_MDB(mode,train_transcriptomics.shape[1]).to(device)
        test_model.load_state_dict(torch.load(f'../../models/PPMI/Multi-modal/{mode}_MDB_{alpha}_best_{seed}.pth'))
    else:
        test_model=multi_modal_model(mode,train_transcriptomics.shape[1]).to(device)
        test_model.load_state_dict(torch.load(f'../../models/PPMI/Multi-modal/{mode}_best_{seed}.pth'))
    test_model.eval()
    # all_probs=[]
    predicted_label=[]
    with torch.no_grad():
        _,_,outputs,_ = test_model(test_transcriptomics_tensor,test_img_tensor)
        # probs = nn.functional.softmax(outputs, dim=1)
        # all_probs.extend(probs.cpu().detach().numpy())
        _, predicted = outputs.max(1)
        predicted_label.extend(predicted.tolist())
    # test_acc = 100.0 * correct / total
    test_acc = f1_score(test_label,predicted_label,average='macro')
    # 计算AUC
    num_classes=4
    true_label_binarized = label_binarize(test_label, classes=list(range(num_classes)))
    # predicted_label_binarized = label_binarize(predicted_label, classes=list(range(num_classes)))
    auc_score = roc_auc_score(true_label_binarized, outputs.cpu().detach().numpy(), average='macro', multi_class='ovr')

    
    print(f'test Acc: {test_acc:.4f},test auc: {auc_score:.4f}')
    print(predicted_label)

    cr, precision, recall, thresholds = calc_confusion_matrix(torch.tensor(predicted_label), test_label_tensor.cpu(), mode, learning_rate, batch_size, epochs)
    # model.apply(weight_init)
    return cr , batch_size, learning_rate, epochs, seed ,auc_score

## Concate

In [ ]:
#embeding=50
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
for s in range(5):
    set_random_seed(s)
    print('seeds:',s)
    cr, bs_, lr_, e_ , seed,auc_score= train('None',False, 32, 250, 0.00001, s,0)
    accurancy.append(cr['accuracy'])
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])
    auc_score_list.append(auc_score)
    print ('-'*55)
print("Mean accuracy is: ",sum(accurancy)/len(accurancy))
print("precision:",sum(precision)/len(precision))
print("recall:",sum(recall)/len(recall))
print("f1:",sum(f1)/len(f1))
print("auc_score:",sum(auc_score_list)/len(auc_score_list))
print("Std accuracy: " + str(np.array(accurancy).std()))
print("Std precision: " + str(np.array(precision).std()))
print("Std recall: " + str(np.array(recall).std()))
print("Std f1: " + str(np.array(f1).std()))
print("Std auc_score: " + str(np.array(auc_score_list).std()))

In [ ]:
test_transcriptomics = pd.read_csv("../../processed_data/PPMI/overlap/X_test_transpot.csv").drop("PATNO Visit", axis=1).values
test_img = make_img("../../processed_data/PPMI/overlap/X_test_img.pkl")
test_label = pd.read_csv("../../processed_data/PPMI/overlap/y_test.csv").drop("PATNO Visit", axis=1).values.astype("int").flatten()
test_transcriptomics_tensor = torch.tensor(test_transcriptomics, dtype=torch.float).to(device)
test_img_tensor = torch.tensor(test_img, dtype=torch.float).to(device)
print('f1_score                 acc                 precision           recall              auc             aupr')
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
aupr_score_list=[]
for i in range(5):
    multi_modal=multi_modal_model('None',test_transcriptomics.shape[1]).to(device)
    multi_modal.load_state_dict(torch.load(f'../../models/PPMI/Multi-modal/None_best_{i}.pth'))
    multi_modal.eval()
    predicted_label_0=[]
    with torch.no_grad():
        _,_,outputs_0,_ = multi_modal(test_transcriptomics_tensor,test_img_tensor)
        # probs_0 = nn.functional.softmax(outputs_0, dim=1)
        # all_probs_0.extend(probs_0.cpu().detach().numpy())
        _, predicted_0 = outputs_0.max(1)
        predicted_label_0.extend(predicted_0.tolist())

    num_classes=4
    true_label_binarized = label_binarize(test_label, classes=list(range(num_classes)))
    test_acc_0=accuracy_score(test_label,predicted_label_0)
    test_f1_0 = f1_score(test_label,predicted_label_0,average='macro')
    test_precision_0=precision_score(test_label,predicted_label_0,average='macro')
    recall_0=recall_score(test_label,predicted_label_0,average='macro')
    auc_score_0 = roc_auc_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro', multi_class='ovr')
    aupr_score_0 = average_precision_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro')
    print(test_f1_0,test_acc_0,test_precision_0,recall_0,auc_score_0,aupr_score_0)
    accurancy.append(test_acc_0)
    precision.append(test_precision_0)
    recall.append(recall_0)
    f1.append(test_f1_0)
    auc_score_list.append(auc_score_0)
    aupr_score_list.append(aupr_score_0)
print('avg')
print(sum(f1)/5,sum(accurancy)/5,sum(precision)/5,sum(recall)/5,sum(auc_score_list)/5,sum(aupr_score_list)/5)

## CMC fusion

In [ ]:
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
# seeds = random.sample(range(1, 200), 5)
for s in range(5):
    set_random_seed(s)
    print('seeds:',s)
    cr, bs_, lr_, e_ , seed,auc_score= train('MM_Tansformer',False, 32, 250, 0.00001, s,0)
    accurancy.append(cr['accuracy'])
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])
    auc_score_list.append(auc_score)
    print ('-'*55)
print("Mean accuracy is: ",sum(accurancy)/len(accurancy))
print("precision:",sum(precision)/len(precision))
print("recall:",sum(recall)/len(recall))
print("f1:",sum(f1)/len(f1))
print("auc_score:",sum(auc_score_list)/len(auc_score_list))
print("Std accuracy: " + str(np.array(accurancy).std()))
print("Std precision: " + str(np.array(precision).std()))
print("Std recall: " + str(np.array(recall).std()))
print("Std f1: " + str(np.array(f1).std()))
print("Std auc_score: " + str(np.array(auc_score_list).std()))

In [ ]:

test_transcriptomics = pd.read_csv("../../processed_data/PPMI/overlap/X_test_transpot.csv").drop("PATNO Visit", axis=1).values
test_img = make_img("../../processed_data/PPMI/overlap/X_test_img.pkl")
test_label = pd.read_csv("../../processed_data/PPMI/overlap/y_test.csv").drop("PATNO Visit", axis=1).values.astype("int").flatten()
test_transcriptomics_tensor = torch.tensor(test_transcriptomics, dtype=torch.float).to(device)
test_img_tensor = torch.tensor(test_img, dtype=torch.float).to(device)

print('f1_score                 acc                 precision           recall              auc             aupr')
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
aupr_score_list=[]
for i in range(5):
    multi_modal=multi_modal_model('MM_Tansformer',test_transcriptomics.shape[1]).to(device)
    multi_modal.load_state_dict(torch.load(f'../../models/PPMI/Multi-modal/MM_Tansformer_best_{i}.pth'))
    multi_modal.eval()
    predicted_label_0=[]
    with torch.no_grad():
        _,_,outputs_0,_ = multi_modal(test_transcriptomics_tensor,test_img_tensor)
        # probs_0 = nn.functional.softmax(outputs_0, dim=1)
        # all_probs_0.extend(probs_0.cpu().detach().numpy())
        _, predicted_0 = outputs_0.max(1)
        predicted_label_0.extend(predicted_0.tolist())

    num_classes=4
    true_label_binarized = label_binarize(test_label, classes=list(range(num_classes)))
    test_acc_0=accuracy_score(test_label,predicted_label_0)
    test_f1_0 = f1_score(test_label,predicted_label_0,average='macro')
    test_precision_0=precision_score(test_label,predicted_label_0,average='macro')
    recall_0=recall_score(test_label,predicted_label_0,average='macro')
    auc_score_0 = roc_auc_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro', multi_class='ovr')
    aupr_score_0 = average_precision_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro')
    print(test_f1_0,test_acc_0,test_precision_0,recall_0,auc_score_0,aupr_score_0)
    accurancy.append(test_acc_0)
    precision.append(test_precision_0)
    recall.append(recall_0)
    f1.append(test_f1_0)
    auc_score_list.append(auc_score_0)
    aupr_score_list.append(aupr_score_0)
print('avg')
print(sum(f1)/5,sum(accurancy)/5,sum(precision)/5,sum(recall)/5,sum(auc_score_list)/5,sum(aupr_score_list)/5)

## MDB

In [ ]:
#embeding=50
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
# seeds = random.sample(range(1, 200), 5)
for s in range(5):
    set_random_seed(s)
    print('seeds:',s)
    cr, bs_, lr_, e_ , seed,auc_score= train('None',True, 32, 250, 0.00001, s,0.2)
    accurancy.append(cr['accuracy'])
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])
    auc_score_list.append(auc_score)
    print ('-'*55)
print("Mean accuracy is: ",sum(accurancy)/len(accurancy))
print("precision:",sum(precision)/len(precision))
print("recall:",sum(recall)/len(recall))
print("f1:",sum(f1)/len(f1))
print("auc_score:",sum(auc_score_list)/len(auc_score_list))
print("Std accuracy: " + str(np.array(accurancy).std()))
print("Std precision: " + str(np.array(precision).std()))
print("Std recall: " + str(np.array(recall).std()))
print("Std f1: " + str(np.array(f1).std()))
print("Std auc_score: " + str(np.array(auc_score_list).std()))

In [ ]:
test_transcriptomics = pd.read_csv("../../processed_data/PPMI/overlap/X_test_transpot.csv").drop("PATNO Visit", axis=1).values
test_img = make_img("../../processed_data/PPMI/overlap/X_test_img.pkl")
test_label = pd.read_csv("../../processed_data/PPMI/overlap/y_test.csv").drop("PATNO Visit", axis=1).values.astype("int").flatten()
test_transcriptomics_tensor = torch.tensor(test_transcriptomics, dtype=torch.float).to(device)
test_img_tensor = torch.tensor(test_img, dtype=torch.float).to(device)
print('f1_score                 acc                 precision           recall              auc             aupr')
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
aupr_score_list=[]
for i in range(5):
    multi_modal=multi_modal_model_MDB('None',test_transcriptomics.shape[1]).to(device)
    multi_modal.load_state_dict(torch.load(f'../../models/PPMI/Multi-modal/None_MDB_0.2_best_{i}.pth'))
    multi_modal.eval()
    predicted_label_0=[]
    with torch.no_grad():
        _,_,outputs_0,_ = multi_modal(test_transcriptomics_tensor,test_img_tensor)
        # probs_0 = nn.functional.softmax(outputs_0, dim=1)
        # all_probs_0.extend(probs_0.cpu().detach().numpy())
        _, predicted_0 = outputs_0.max(1)
        predicted_label_0.extend(predicted_0.tolist())

    num_classes=4
    true_label_binarized = label_binarize(test_label, classes=list(range(num_classes)))
    test_acc_0=accuracy_score(test_label,predicted_label_0)
    test_f1_0 = f1_score(test_label,predicted_label_0,average='macro')
    test_precision_0=precision_score(test_label,predicted_label_0,average='macro')
    recall_0=recall_score(test_label,predicted_label_0,average='macro')
    auc_score_0 = roc_auc_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro', multi_class='ovr')
    aupr_score_0 = average_precision_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro')
    print(test_f1_0,test_acc_0,test_precision_0,recall_0,auc_score_0,aupr_score_0)
    accurancy.append(test_acc_0)
    precision.append(test_precision_0)
    recall.append(recall_0)
    f1.append(test_f1_0)
    auc_score_list.append(auc_score_0)
    aupr_score_list.append(aupr_score_0)
print('avg')
print(sum(f1)/5,sum(accurancy)/5,sum(precision)/5,sum(recall)/5,sum(auc_score_list)/5,sum(aupr_score_list)/5)

## CMC fusion+MDB

In [ ]:
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
for s in range(5):
    set_random_seed(s)
    print('seeds:',s)
    cr, bs_, lr_, e_ , seed,auc_score= train('MM_Tansformer',True, 32, 250, 0.00001, s,0.2)
    accurancy.append(cr['accuracy'])
    precision.append(cr["macro avg"]["precision"])
    recall.append(cr["macro avg"]["recall"])
    f1.append(cr["macro avg"]["f1-score"])
    auc_score_list.append(auc_score)
    print ('-'*55)
print("Mean accuracy is: ",sum(accurancy)/len(accurancy))
print("precision:",sum(precision)/len(precision))
print("recall:",sum(recall)/len(recall))
print("f1:",sum(f1)/len(f1))
print("auc_score:",sum(auc_score_list)/len(auc_score_list))
print("Std accuracy: " + str(np.array(accurancy).std()))
print("Std precision: " + str(np.array(precision).std()))
print("Std recall: " + str(np.array(recall).std()))
print("Std f1: " + str(np.array(f1).std()))
print("Std auc_score: " + str(np.array(auc_score_list).std()))

In [ ]:

test_transcriptomics = pd.read_csv("../../processed_data/PPMI/overlap/X_test_transpot.csv").drop("PATNO Visit", axis=1).values
test_img = make_img("../../processed_data/PPMI/overlap/X_test_img.pkl")
test_label = pd.read_csv("../../processed_data/PPMI/overlap/y_test.csv").drop("PATNO Visit", axis=1).values.astype("int").flatten()
test_transcriptomics_tensor = torch.tensor(test_transcriptomics, dtype=torch.float).to(device)
test_img_tensor = torch.tensor(test_img, dtype=torch.float).to(device)
print('f1_score                 acc                 precision           recall              auc             aupr')
accurancy=[]
precision=[]
recall=[]
f1=[]
auc_score_list=[]
aupr_score_list=[]
for i in range(5):
    multi_modal=multi_modal_model_MDB('MM_Tansformer',test_transcriptomics.shape[1]).to(device)
    multi_modal.load_state_dict(torch.load(f'../../models/PPMI/Multi-modal/MM_Tansformer_MDB_0.2_best_{i}.pth'))
    multi_modal.eval()
    predicted_label_0=[]
    with torch.no_grad():
        _,_,outputs_0,_ = multi_modal(test_transcriptomics_tensor,test_img_tensor)
        # probs_0 = nn.functional.softmax(outputs_0, dim=1)
        # all_probs_0.extend(probs_0.cpu().detach().numpy())
        _, predicted_0 = outputs_0.max(1)
        predicted_label_0.extend(predicted_0.tolist())

    num_classes=4
    true_label_binarized = label_binarize(test_label, classes=list(range(num_classes)))
    test_acc_0=accuracy_score(test_label,predicted_label_0)
    test_f1_0 = f1_score(test_label,predicted_label_0,average='macro')
    test_precision_0=precision_score(test_label,predicted_label_0,average='macro')
    recall_0=recall_score(test_label,predicted_label_0,average='macro')
    auc_score_0 = roc_auc_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro', multi_class='ovr')
    aupr_score_0 = average_precision_score(true_label_binarized, outputs_0.cpu().detach().numpy(), average='macro')
    print(test_f1_0,test_acc_0,test_precision_0,recall_0,auc_score_0,aupr_score_0)
    accurancy.append(test_acc_0)
    precision.append(test_precision_0)
    recall.append(recall_0)
    f1.append(test_f1_0)
    auc_score_list.append(auc_score_0)
    aupr_score_list.append(aupr_score_0)
print('avg')
print(sum(f1)/5,sum(accurancy)/5,sum(precision)/5,sum(recall)/5,sum(auc_score_list)/5,sum(aupr_score_list)/5)